# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [4]:
import pandas as pd
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [31]:
df['action'].value_counts()

view     6328
click    1860
Name: action, dtype: int64

4468 viewed without clicking. Given that, in most cases, a view is neccessary for a click, we can safely assume that there were no clicks without views.

In [32]:
overlap_count = 0
for i in df['id'].value_counts():
    if i >1:
        overlap_count += 1
print(overlap_count)

1860


In [33]:
ex_ids = set(df[df.group=='experiment']['id'].unique())

In [34]:
ctrl_ids = set(df[df.group=='control']['id'].unique())

In [35]:
for ID in ex_ids:
    if ID in ctrl_ids:
        print(ID)

We get an empty output, so there is no overlap between the two groups.

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [36]:
df['count'] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [40]:
ctrl = df[df.group=='control'].pivot(index='id', columns='action', values='count')
ctrl = ctrl.fillna(value=0)

In [49]:
exp = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
exp = exp.fillna(value=0)

In [51]:
display(ctrl,exp)

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
...,...,...
936786,0.0,1.0
937003,0.0,1.0
937073,0.0,1.0


action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
...,...,...
935382,0.0,1.0
935576,0.0,1.0
935742,1.0,1.0


In [63]:
import numpy as np
import scipy.stats as stats
def welch_t(a, b):
    num = a.mean() - b.mean()
    denom = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    return np.abs(num/denom)

def welch_df(a, b):
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    num = (s1/n1 + s2/n2)**2
    denom = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    return num/denom

def p_value(a, b, two_sided=False):
    t = welch_t(a, b)
    df = welch_df(a, b)
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [66]:
p_value(exp.click, ctrl.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [71]:
ctrl_mean = ctrl.click.mean()
expected_clicks = ctrl_mean * len(exp)
expected_clicks 

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [79]:
#𝑛∙𝑝(1−𝑝) 
variance = len(exp) * (ctrl_mean * (1-ctrl_mean))
variance

603.6135462588397

In [80]:
#sqrt of variance
std = np.sqrt(variance)
std

24.568547907005815

In [85]:
#Z = (actual-expected/std)
sum_exp_clicks = exp.click.sum()
Z = (sum_exp_clicks - expected_clicks)/std
Z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [89]:
import scipy.stats as stats
p = stats.norm.sf(Z)
p

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: This result is a lower p-value than the previous statistical test, but both are low enough to reject the null hypothesis.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.